---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [ ]:
import pandas as pd


# Create a Series from the content of dates.txt:
with open('dates.txt') as file:
    doc = [note for note in file]
df = pd.Series(doc)

df.head(10)

In [ ]:
def date_sorter():
    """Build a Series containing the indices of the notes in dates.txt. These
       indices are organized so that the corresponding dates are in chronological order.

    Returns:
        indices: Indices of the notes, organized so that the dates are in chronological order.
    """

    # Columns for storing the dates:
    date_cols = ['Year', 'Month', 'Day']

    # Months dictionary:
    mon_dic = {
        'jan': '01',
        'feb': '02',
        'mar': '03',
        'apr': '04',
        'may': '05',
        'jun': '06',
        'jul': '07',
        'aug': '08',
        'sep': '09',
        'oct': '10',
        'nov': '11',
        'dec': '12'
    }

    # Create the list of regular expressions that will be used to extract the dates from the notes:
    days = '|'.join([str(day) for day in range(1, 32)] + ['0' + str(day) for day in range(1, 10)])
    mons = '|'.join(list(mon_dic.values()) + [str(month) for month in range(1, 10)])
    regex_day = r'(?P<Day>' + days + r')'
    regex_month_1 = r'(?P<Month>' + '|'.join(list(mon_dic.keys())) + r')'
    regex_month_2 = r'(?P<Month>' + mons + r')'
    regex_year_1 = r'(?P<Year>(?<![0-9])(?:(?:[789]{1}[0-9]{1})|(?:(?:19|20){1}[0-9]{2}))(?![0-9]))'
    regex_year_2 = r'(?P<Year>(?<![0-9])(?:(?:19|20){1}[0-9]{2})(?![0-9]))'
    regexs = [
        r'(?P<Date>' + regex_day + r' +' + regex_month_1 + r'[a-z.,]* +' + regex_year_1 + r')',
        r'(?P<Date>' + regex_month_1 + r'[a-z.,]*[ -]+' + regex_day + r'[a-z,]*[ -]+' + regex_year_1 + r')',
        r'(?P<Date>' + regex_month_2 + r'[-/]+' + regex_day + r'[-/]+' + regex_year_1 + r')',
        r'(?P<Date>' + regex_month_1 + r'[a-z.,]*[ -]+(?:' + regex_day + r'[a-z,]*[ -]+)?' + regex_year_1 + r')',
        r'(?P<Date>' + regex_month_2 + r'[-/]+(?:' + regex_day + r'[-/]+)?' + regex_year_1 + r')',
        r'(?P<Date>(?:' + regex_month_2 + r'[-/]+(?:' + regex_day + r'[-/]+)?)?' + regex_year_2 + r')'
    ]


    def extract_fix_dates(notes, regex):
        """Build a DataFrame containing the dates.

        Arguments:
            notes: Series with notes that contain dates.
            regex: String corresponding to the regular expression used to extract the dates.

        Returns:
            dates: DataFrame with the dates extracted from notes.
        """

        # Convert every note to lowercase, and extract the dates:
        dates = notes.str.lower().str.extract(regex)

        # Fix the dates with missing values in the Month and Day columns:
        dates[date_cols[1:]] = dates[date_cols[1:]].fillna("1")

        # Remove every row that wasn't completely fixed by the previous command:
        dates = dates.dropna()

        # Use the mon_dic dictionary to map the month names to numbers (as strings):
        dates['Month'] = dates['Month'].apply(lambda m: mon_dic[m] if m in mon_dic else m)

        # Convert the values in the Month, Day and Year columns to integers:
        dates[date_cols] = dates[date_cols].astype(int)

        # Fix the years with only 2 digits:
        dates['Year'] = dates['Year'].apply(lambda y: y + 1900 if y < 100 else y)

        return dates


    def extract_fix_sort_dates(notes, regexs):
        """Build a DataFrame containing the sorted dates. The dates are extracted
           with the aid of several regular expressions.

        Arguments:
            notes: Series with notes that contain dates.
            regexs: List of strings corresponding to the regular expressions used to extract the dates.

        Returns:
            sorted_dates: DataFrame with the dates extracted from notes. These dates are sorted in ascending order.
            remain_notes: Series with the notes containing dates that couldn't be extracted.
        """

        # Initialize remain_notes:
        remain_notes = notes.copy()

        # List of DataFrames containing dates:
        dates_dfs = []

        # Loop over the regular expressions:
        for regex in regexs:
            # Use the regular expression regex to extract dates from the remaining notes:
            tmp_df = extract_fix_dates(remain_notes, regex)
            # Update dates_dfs:
            dates_dfs.append(tmp_df)
            # Update the remaining notes:
            remain_notes = remain_notes.drop(tmp_df.index.values)

        # Concatenate the DataFrames stored in dates_dfs, and sort the dates:
        sorted_dates = pd.concat(dates_dfs).sort_values(date_cols)

        return sorted_dates, remain_notes


    # Extract the dates from the notes, and put these dates in chronological order:
    sorted_dates, remain_notes = extract_fix_sort_dates(df, regexs)

    # Build the Series containing the indices of the notes:
    indices = pd.Series(data=sorted_dates.index.values, dtype=int)

    # A few tests:
    num_notes = df.size
    assert indices.size == num_notes
    assert remain_notes.size == 0
    assert sorted_dates['Day'].between(1, 31).astype(int).sum() == num_notes
    assert sorted_dates['Month'].between(1, 12).astype(int).sum() == num_notes
    assert sorted_dates['Year'].between(1970, 2016).astype(int).sum() == num_notes

    return indices